# LAB 2.5: ADVANCED PROMPT CHAINING WITH CONTEXT

**Course:** Advanced Prompt Engineering Training  
**Session:** Session 2 - Advanced Context Engineering  
**Duration:** 50 minutes  
**Difficulty:** ⭐⭐⭐⭐⭐  
**Type:** Hands-on Multi-Step Workflow Orchestration

## LAB OVERVIEW

This lab focuses on **chaining multiple LLM calls** where each step refines context and passes results to the next. You'll learn to:

- Build sequential processing chains
- Implement parallel execution for efficiency
- Create conditional branching logic
- Design context refinement workflows
- Orchestrate production multi-step pipelines

**Scenario:** You're building a loan underwriting workflow. The process requires multiple steps:

1. **Extract** key information from application
2. **Verify** data consistency across documents
3. **Calculate** financial ratios (DTI, DSCR, LTV)
4. **Assess** risk based on calculations
5. **Generate** final recommendation with explanation

**Challenge:** Each step depends on previous steps, but you can't fit everything in one prompt. Use chaining to break down the complex task while maintaining context.

## LEARNING OBJECTIVES

By the end of this lab, you will be able to:

✓ Design sequential prompt chains  
✓ Implement parallel processing for speed  
✓ Build conditional branching logic  
✓ Create context refinement workflows  
✓ Orchestrate production chain pipelines  
✓ Handle errors and retries in chains

## SETUP INSTRUCTIONS

### Step 1: Import Libraries

In [ ]:
# Lab 2.5: Advanced Prompt Chaining with Context

import os
import json
from openai import OpenAI
import tiktoken
import pandas as pd
from typing import Dict, List, Any, Optional, Callable
from datetime import datetime
from enum import Enum
from dotenv import load_dotenv

load_dotenv(override=True)

print("✓ Libraries imported")

### Step 2: Configure Client

In [ ]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
encoding = tiktoken.encoding_for_model("gpt-4")
MODEL = "gpt-4"

def count_tokens(text: str) -> int:
    return len(encoding.encode(text))

def call_gpt4(prompt: str, system_prompt: str = "You are a helpful AI assistant.") -> Dict:
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        return {
            "content": response.choices[0].message.content,
            "total_tokens": response.usage.total_tokens,
            "success": True
        }
    except Exception as e:
        return {"content": "", "error": str(e), "success": False}

print(f"✓ Model: {MODEL}")

### Step 3: Load Sample Data

In [ ]:
# Loan application data for testing chains

loan_application = {
    "applicant": "Martinez Tech Ventures LLC",
    "loan_amount": 500000,
    "property_value": 800000,
    "annual_revenue": 850000,
    "annual_expenses": 680000,
    "existing_debt_payment": 4000,
    "credit_score": 720,
    "years_in_business": 4,
    "industry": "Software Development"
}

print("✓ Sample data loaded")
print(f"\nApplication Summary:")
print(f"  Applicant: {loan_application['applicant']}")
print(f"  Loan Amount: ${loan_application['loan_amount']:,}")
print(f"  Property Value: ${loan_application['property_value']:,}")
print(f"  Credit Score: {loan_application['credit_score']}")

## PROMPT CHAINING FUNDAMENTALS

### Why Chain Prompts?

**Single Monolithic Prompt:**
```python
# BAD: Everything in one massive prompt
prompt = f"""
Extract info, verify consistency, calculate ratios, assess risk, 
generate recommendation for: {all_data}
"""
# Problem: 10,000+ tokens, unclear logic, hard to debug
```

**Chained Prompts:**
```python
# GOOD: Break into steps
step1 = extract_information(data)
step2 = verify_consistency(step1)
step3 = calculate_ratios(step2)
step4 = assess_risk(step3)
step5 = generate_recommendation(step4)
# Benefit: Clear, debuggable, efficient
```

### Chain Types

| Type | Description | Use Case |
|------|-------------|----------|
| **Sequential** | A → B → C → D | Linear workflows |
| **Parallel** | A → [B, C, D] → E | Independent tasks |
| **Conditional** | A → if X then B else C | Decision trees |
| **Refinement** | A → B → refine(A,B) → C | Iterative improvement |

## CHALLENGE 1: SEQUENTIAL CHAINING

**Time:** 10 minutes  
**Difficulty:** ⭐⭐⭐☆☆  
**Objective:** Build a linear multi-step chain

### Background

Sequential chaining passes output from one step as input to the next.

In [ ]:
# SOLUTION: Sequential Chain

class SequentialChain:
    """Linear prompt chain"""
    
    def __init__(self):
        self.steps_executed = []
        self.total_tokens = 0
    
    def step_extract(self, data: Dict) -> Dict:
        """Step 1: Extract key information"""
        prompt = f"""
Extract key financial metrics from this loan application.
Return as JSON with: loan_amount, ltv_ratio, annual_revenue, annual_profit

Application: {json.dumps(data, indent=2)}

Return ONLY valid JSON.
"""
        result = call_gpt4(prompt, "You are a data extraction specialist.")
        self.total_tokens += result.get('total_tokens', 0)
        
        if result['success']:
            try:
                # Parse JSON from response
                import re
                json_match = re.search(r'\{.*\}', result['content'], re.DOTALL)
                extracted = json.loads(json_match.group()) if json_match else {}
                
                self.steps_executed.append({
                    'step': 'extract',
                    'tokens': result['total_tokens'],
                    'output': extracted
                })
                return extracted
            except:
                return {}
        return {}
    
    def step_calculate(self, extracted: Dict, original: Dict) -> Dict:
        """Step 2: Calculate financial ratios"""
        prompt = f"""
Calculate these ratios:
1. LTV (Loan-to-Value) = loan_amount / property_value
2. DSCR (Debt Service Coverage) = (revenue - expenses) / (debt_payment * 12)
3. Profit Margin = (revenue - expenses) / revenue

Data:
- Loan: ${original['loan_amount']}
- Property: ${original['property_value']}
- Revenue: ${original['annual_revenue']}
- Expenses: ${original['annual_expenses']}
- Monthly Debt: ${original['existing_debt_payment']}

Return as JSON: {{"ltv": 0.xx, "dscr": 0.xx, "profit_margin": 0.xx}}
"""
        result = call_gpt4(prompt, "You are a financial analyst.")
        self.total_tokens += result.get('total_tokens', 0)
        
        if result['success']:
            try:
                import re
                json_match = re.search(r'\{.*\}', result['content'], re.DOTALL)
                ratios = json.loads(json_match.group()) if json_match else {}
                
                self.steps_executed.append({
                    'step': 'calculate',
                    'tokens': result['total_tokens'],
                    'output': ratios
                })
                return ratios
            except:
                return {}
        return {}
    
    def step_assess(self, ratios: Dict, original: Dict) -> str:
        """Step 3: Assess risk level"""
        prompt = f"""
Assess loan risk based on these metrics:

Ratios:
- LTV: {ratios.get('ltv', 0):.1%} (target: <75%)
- DSCR: {ratios.get('dscr', 0):.2f}x (target: >1.25x)
- Profit Margin: {ratios.get('profit_margin', 0):.1%} (target: >10%)

Applicant:
- Credit Score: {original['credit_score']} (min: 680)
- Years in Business: {original['years_in_business']} (min: 2)

Classify as: LOW_RISK, MEDIUM_RISK, or HIGH_RISK

Return one word only.
"""
        result = call_gpt4(prompt, "You are a risk analyst.")
        self.total_tokens += result.get('total_tokens', 0)
        
        risk = result['content'].strip() if result['success'] else "UNKNOWN"
        
        self.steps_executed.append({
            'step': 'assess',
            'tokens': result['total_tokens'],
            'output': risk
        })
        return risk
    
    def step_recommend(self, risk: str, ratios: Dict) -> str:
        """Step 4: Generate recommendation"""
        prompt = f"""
Generate a loan recommendation.

Risk Level: {risk}
LTV: {ratios.get('ltv', 0):.1%}
DSCR: {ratios.get('dscr', 0):.2f}x

Recommendation should include:
1. Decision (APPROVE, APPROVE_WITH_CONDITIONS, DENY)
2. Reasoning (2-3 sentences)
3. Conditions (if applicable)

Be concise and professional.
"""
        result = call_gpt4(prompt, "You are a senior underwriter.")
        self.total_tokens += result.get('total_tokens', 0)
        
        recommendation = result['content'] if result['success'] else "Unable to generate recommendation"
        
        self.steps_executed.append({
            'step': 'recommend',
            'tokens': result['total_tokens'],
            'output': recommendation
        })
        return recommendation
    
    def execute(self, data: Dict) -> Dict:
        """Execute full chain"""
        # Step 1: Extract
        extracted = self.step_extract(data)
        
        # Step 2: Calculate
        ratios = self.step_calculate(extracted, data)
        
        # Step 3: Assess
        risk = self.step_assess(ratios, data)
        
        # Step 4: Recommend
        recommendation = self.step_recommend(risk, ratios)
        
        return {
            'extracted': extracted,
            'ratios': ratios,
            'risk': risk,
            'recommendation': recommendation,
            'steps_executed': len(self.steps_executed),
            'total_tokens': self.total_tokens
        }

print("✓ SequentialChain class defined")

### Test Sequential Chain

In [ ]:
# Test sequential chain
print("SEQUENTIAL CHAIN:")
print("=" * 80)

chain = SequentialChain()
result = chain.execute(loan_application)

print(f"\nAPPLICATION: {loan_application['applicant']}")
print(f"\nRATIOS:")
for key, value in result['ratios'].items():
    if isinstance(value, float):
        print(f"  {key.upper()}: {value:.2%}" if value < 1 else f"  {key.upper()}: {value:.2f}x")

print(f"\nRISK LEVEL: {result['risk']}")
print(f"\nRECOMMENDATION:")
print(result['recommendation'])

print(f"\nCHAIN METRICS:")
print(f"  Steps: {result['steps_executed']}")
print(f"  Total Tokens: {result['total_tokens']}")

print("=" * 80)

## CHALLENGE 2: PARALLEL PROCESSING CHAINS

**Time:** 10 minutes  
**Difficulty:** ⭐⭐⭐⭐☆  
**Objective:** Execute independent tasks in parallel

### Background

Some tasks don't depend on each other and can run simultaneously for speed.

In [ ]:
# SOLUTION: Parallel Processing

import concurrent.futures

class ParallelChain:
    """Execute independent tasks in parallel"""
    
    def __init__(self):
        self.total_tokens = 0
    
    def analyze_credit(self, data: Dict) -> Dict:
        """Parallel task 1: Credit analysis"""
        prompt = f"""
Analyze creditworthiness:
- Credit Score: {data['credit_score']}
- Years in Business: {data['years_in_business']}
- Industry: {data['industry']}

Return: STRONG, ACCEPTABLE, or WEAK
"""
        result = call_gpt4(prompt)
        return {
            'task': 'credit',
            'assessment': result['content'].strip() if result['success'] else 'UNKNOWN',
            'tokens': result.get('total_tokens', 0)
        }
    
    def analyze_financials(self, data: Dict) -> Dict:
        """Parallel task 2: Financial analysis"""
        prompt = f"""
Analyze financial strength:
- Revenue: ${data['annual_revenue']}
- Expenses: ${data['annual_expenses']}
- Profit: ${data['annual_revenue'] - data['annual_expenses']}

Return: STRONG, ACCEPTABLE, or WEAK
"""
        result = call_gpt4(prompt)
        return {
            'task': 'financials',
            'assessment': result['content'].strip() if result['success'] else 'UNKNOWN',
            'tokens': result.get('total_tokens', 0)
        }
    
    def analyze_collateral(self, data: Dict) -> Dict:
        """Parallel task 3: Collateral analysis"""
        prompt = f"""
Analyze collateral adequacy:
- Loan: ${data['loan_amount']}
- Property Value: ${data['property_value']}
- LTV: {data['loan_amount']/data['property_value']:.1%}

Return: STRONG, ACCEPTABLE, or WEAK
"""
        result = call_gpt4(prompt)
        return {
            'task': 'collateral',
            'assessment': result['content'].strip() if result['success'] else 'UNKNOWN',
            'tokens': result.get('total_tokens', 0)
        }
    
    def synthesize(self, parallel_results: List[Dict]) -> str:
        """Combine parallel results"""
        assessments = {r['task']: r['assessment'] for r in parallel_results}
        
        prompt = f"""
Synthesize overall assessment from these analyses:

Credit: {assessments.get('credit', 'UNKNOWN')}
Financials: {assessments.get('financials', 'UNKNOWN')}
Collateral: {assessments.get('collateral', 'UNKNOWN')}

Provide overall recommendation (APPROVE/DENY) with reasoning.
"""
        result = call_gpt4(prompt)
        return result['content'] if result['success'] else 'Unable to synthesize'
    
    def execute(self, data: Dict) -> Dict:
        """Execute parallel tasks"""
        import time
        start = time.time()
        
        # Execute parallel tasks
        with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
            futures = {
                executor.submit(self.analyze_credit, data): 'credit',
                executor.submit(self.analyze_financials, data): 'financials',
                executor.submit(self.analyze_collateral, data): 'collateral'
            }
            
            parallel_results = []
            for future in concurrent.futures.as_completed(futures):
                parallel_results.append(future.result())
        
        parallel_time = time.time() - start
        
        # Synthesize results
        synthesis = self.synthesize(parallel_results)
        
        total_time = time.time() - start
        total_tokens = sum(r['tokens'] for r in parallel_results)
        
        return {
            'parallel_results': parallel_results,
            'synthesis': synthesis,
            'parallel_time': parallel_time,
            'total_time': total_time,
            'total_tokens': total_tokens
        }

print("✓ ParallelChain class defined")

### Test Parallel Chain

In [ ]:
# Test parallel chain
print("PARALLEL PROCESSING CHAIN:")
print("=" * 80)

parallel = ParallelChain()
result = parallel.execute(loan_application)

print(f"\nPARALLEL ANALYSES:")
for analysis in result['parallel_results']:
    print(f"  {analysis['task'].upper()}: {analysis['assessment']}")

print(f"\nSYNTHESIS:")
print(result['synthesis'])

print(f"\nPERFORMANCE:")
print(f"  Parallel execution: {result['parallel_time']:.2f}s")
print(f"  Total time: {result['total_time']:.2f}s")
print(f"  Total tokens: {result['total_tokens']}")

print("=" * 80)

## CHALLENGE 3: CONDITIONAL BRANCHING

**Time:** 10 minutes  
**Difficulty:** ⭐⭐⭐⭐☆  
**Objective:** Build decision trees in chains

### Background

Different inputs require different processing paths.

In [ ]:
# SOLUTION: Conditional Branching

class ConditionalChain:
    """Chain with conditional branching logic"""
    
    def __init__(self):
        self.execution_path = []
        self.total_tokens = 0
    
    def classify_application(self, data: Dict) -> str:
        """Classify application type"""
        loan_size = data['loan_amount']
        
        if loan_size < 100000:
            category = "SMALL_BUSINESS"
        elif loan_size < 500000:
            category = "MEDIUM_BUSINESS"
        else:
            category = "LARGE_COMMERCIAL"
        
        self.execution_path.append(f"Classified as: {category}")
        return category
    
    def process_small_business(self, data: Dict) -> str:
        """Simplified process for small loans"""
        prompt = f"""
Quick assessment for small business loan:
- Amount: ${data['loan_amount']}
- Credit: {data['credit_score']}
- Business Age: {data['years_in_business']} years

Approve if credit > 680 and business age > 2 years.
Return: APPROVED or DENIED with brief reason.
"""
        result = call_gpt4(prompt)
        self.execution_path.append("Used simplified process")
        self.total_tokens += result.get('total_tokens', 0)
        return result['content'] if result['success'] else 'Error'
    
    def process_medium_business(self, data: Dict) -> str:
        """Standard process for medium loans"""
        # Step 1: Financial check
        prompt1 = f"""
Check financial viability:
Revenue: ${data['annual_revenue']}
Expenses: ${data['annual_expenses']}
Debt: ${data['existing_debt_payment']}/month

Is DSCR > 1.25? Return YES or NO.
"""
        result1 = call_gpt4(prompt1)
        self.total_tokens += result1.get('total_tokens', 0)
        
        viable = 'YES' in result1['content'].upper() if result1['success'] else False
        
        if not viable:
            self.execution_path.append("Failed financial check")
            return "DENIED: Insufficient debt service coverage"
        
        # Step 2: Full assessment
        prompt2 = f"""
Full assessment for ${data['loan_amount']} loan.
Credit: {data['credit_score']}, Business: {data['years_in_business']}yr
Financials: Viable

Recommendation with conditions?
"""
        result2 = call_gpt4(prompt2)
        self.execution_path.append("Completed full assessment")
        self.total_tokens += result2.get('total_tokens', 0)
        return result2['content'] if result2['success'] else 'Error'
    
    def process_large_commercial(self, data: Dict) -> str:
        """Enhanced process for large loans"""
        # Step 1: Initial screening
        if data['credit_score'] < 700:
            self.execution_path.append("Failed credit threshold")
            return "DENIED: Large commercial loans require credit score ≥ 700"
        
        # Step 2: Detailed analysis
        prompt = f"""
Comprehensive analysis for ${data['loan_amount']:,} commercial loan:

Financials:
- Revenue: ${data['annual_revenue']:,}
- Expenses: ${data['annual_expenses']:,}  
- Profit: ${data['annual_revenue'] - data['annual_expenses']:,}

Credit: {data['credit_score']}
Property: ${data['property_value']:,}

Provide detailed recommendation with:
1. Approval decision
2. Required conditions
3. Interest rate recommendation
4. Monitoring requirements
"""
        result = call_gpt4(prompt)
        self.execution_path.append("Completed enhanced due diligence")
        self.total_tokens += result.get('total_tokens', 0)
        return result['content'] if result['success'] else 'Error'
    
    def execute(self, data: Dict) -> Dict:
        """Execute conditional chain"""
        # Classify
        category = self.classify_application(data)
        
        # Branch based on classification
        if category == "SMALL_BUSINESS":
            result = self.process_small_business(data)
        elif category == "MEDIUM_BUSINESS":
            result = self.process_medium_business(data)
        else:
            result = self.process_large_commercial(data)
        
        return {
            'category': category,
            'result': result,
            'execution_path': self.execution_path,
            'total_tokens': self.total_tokens
        }

print("✓ ConditionalChain class defined")

### Test Conditional Chain

In [ ]:
# Test conditional chain
print("CONDITIONAL BRANCHING CHAIN:")
print("=" * 80)

# Test with different loan sizes
test_cases = [
    {**loan_application, 'loan_amount': 75000, 'applicant': 'Small Loan'},
    {**loan_application, 'loan_amount': 300000, 'applicant': 'Medium Loan'},
    {**loan_application, 'loan_amount': 1000000, 'applicant': 'Large Loan'}
]

for test_data in test_cases:
    print(f"\n{test_data['applicant']} (${test_data['loan_amount']:,}):")
    print("-" * 80)
    
    chain = ConditionalChain()
    result = chain.execute(test_data)
    
    print(f"Category: {result['category']}")
    print(f"Path: {' → '.join(result['execution_path'])}")
    print(f"Result: {result['result'][:150]}...")
    print(f"Tokens: {result['total_tokens']}")

print("\n" + "=" * 80)

## CHALLENGE 4: CONTEXT REFINEMENT CHAINS

**Time:** 10 minutes  
**Difficulty:** ⭐⭐⭐⭐☆  
**Objective:** Iteratively refine output quality

### Background

Some tasks benefit from refinement: initial draft → critique → improved version.

In [ ]:
# SOLUTION: Context Refinement Chain

class RefinementChain:
    """Iterative refinement workflow"""
    
    def __init__(self, max_iterations: int = 2):
        self.max_iterations = max_iterations
        self.iterations = []
    
    def generate_initial(self, data: Dict) -> str:
        """Generate initial recommendation"""
        prompt = f"""
Generate loan recommendation for:
- Applicant: {data['applicant']}
- Amount: ${data['loan_amount']:,}
- Credit: {data['credit_score']}
- LTV: {data['loan_amount']/data['property_value']:.1%}

Provide recommendation.
"""
        result = call_gpt4(prompt)
        output = result['content'] if result['success'] else ''
        
        self.iterations.append({
            'iteration': 0,
            'type': 'initial',
            'output': output,
            'tokens': result.get('total_tokens', 0)
        })
        return output
    
    def critique(self, recommendation: str) -> Dict:
        """Critique current version"""
        prompt = f"""
Critique this loan recommendation:

{recommendation}

Identify:
1. Missing information
2. Unclear statements
3. Areas needing more detail

Return JSON: {{"issues": ["issue1", "issue2", ...], "score": 0-10}}
"""
        result = call_gpt4(prompt)
        
        if result['success']:
            try:
                import re
                json_match = re.search(r'\{.*\}', result['content'], re.DOTALL)
                critique = json.loads(json_match.group()) if json_match else {'issues': [], 'score': 5}
            except:
                critique = {'issues': [], 'score': 5}
        else:
            critique = {'issues': [], 'score': 5}
        
        self.iterations.append({
            'type': 'critique',
            'output': critique,
            'tokens': result.get('total_tokens', 0)
        })
        return critique
    
    def refine(self, original: str, critique: Dict) -> str:
        """Refine based on critique"""
        prompt = f"""
Improve this recommendation based on critique:

ORIGINAL:
{original}

ISSUES TO ADDRESS:
{json.dumps(critique.get('issues', []), indent=2)}

Provide improved version.
"""
        result = call_gpt4(prompt)
        output = result['content'] if result['success'] else original
        
        self.iterations.append({
            'type': 'refinement',
            'output': output,
            'tokens': result.get('total_tokens', 0)
        })
        return output
    
    def execute(self, data: Dict) -> Dict:
        """Execute refinement chain"""
        # Initial generation
        output = self.generate_initial(data)
        
        # Refinement iterations
        for i in range(self.max_iterations):
            critique = self.critique(output)
            
            # Stop if quality is good enough
            if critique.get('score', 0) >= 8:
                break
            
            output = self.refine(output, critique)
        
        total_tokens = sum(iter['tokens'] for iter in self.iterations)
        
        return {
            'final_output': output,
            'iterations': len([i for i in self.iterations if i['type'] != 'critique']),
            'refinement_steps': self.iterations,
            'total_tokens': total_tokens
        }

print("✓ RefinementChain class defined")

### Test Refinement Chain

In [ ]:
# Test refinement chain
print("CONTEXT REFINEMENT CHAIN:")
print("=" * 80)

refiner = RefinementChain(max_iterations=2)
result = refiner.execute(loan_application)

print(f"\nREFINEMENT PROCESS:")
for i, step in enumerate(result['refinement_steps']):
    print(f"\nStep {i+1}: {step['type'].upper()}")
    if step['type'] == 'critique':
        print(f"  Score: {step['output'].get('score', 'N/A')}/10")
        print(f"  Issues: {len(step['output'].get('issues', []))}")
    else:
        print(f"  Output: {str(step['output'])[:100]}...")
    print(f"  Tokens: {step['tokens']}")

print(f"\nFINAL OUTPUT:")
print(result['final_output'])

print(f"\nMETRICS:")
print(f"  Refinement iterations: {result['iterations']}")
print(f"  Total tokens: {result['total_tokens']}")

print("=" * 80)

## CHALLENGE 5: PRODUCTION CHAIN ORCHESTRATOR

**Time:** 10 minutes  
**Difficulty:** ⭐⭐⭐⭐⭐  
**Objective:** Build complete chain management system

In [ ]:
# SOLUTION: Production Chain Orchestrator

from enum import Enum

class ChainType(Enum):
    SEQUENTIAL = "sequential"
    PARALLEL = "parallel"
    CONDITIONAL = "conditional"
    REFINEMENT = "refinement"

class ChainOrchestrator:
    """Production chain management system"""
    
    def __init__(self):
        self.chains = {
            ChainType.SEQUENTIAL: SequentialChain(),
            ChainType.PARALLEL: ParallelChain(),
            ChainType.CONDITIONAL: ConditionalChain(),
            ChainType.REFINEMENT: RefinementChain()
        }
        self.execution_log = []
    
    def execute_chain(
        self,
        chain_type: ChainType,
        data: Dict,
        **kwargs
    ) -> Dict:
        """Execute specified chain type"""
        import time
        start = time.time()
        
        chain = self.chains[chain_type]
        result = chain.execute(data)
        
        execution_time = time.time() - start
        
        log_entry = {
            'chain_type': chain_type.value,
            'timestamp': datetime.now().isoformat(),
            'execution_time': execution_time,
            'success': True,
            'tokens': result.get('total_tokens', 0)
        }
        self.execution_log.append(log_entry)
        
        result['execution_time'] = execution_time
        result['chain_type'] = chain_type.value
        
        return result
    
    def get_stats(self) -> Dict:
        """Get orchestrator statistics"""
        if not self.execution_log:
            return {'total_executions': 0}
        
        return {
            'total_executions': len(self.execution_log),
            'total_tokens': sum(log['tokens'] for log in self.execution_log),
            'avg_execution_time': sum(log['execution_time'] for log in self.execution_log) / len(self.execution_log),
            'chains_used': list(set(log['chain_type'] for log in self.execution_log))
        }

print("✓ ChainOrchestrator class defined")

### Test Orchestrator

In [ ]:
# Test orchestrator
print("PRODUCTION CHAIN ORCHESTRATOR:")
print("=" * 80)

orchestrator = ChainOrchestrator()

# Execute different chains
print("\nExecuting chains...")

results = {}
for chain_type in ChainType:
    print(f"\n  {chain_type.value}...", end='')
    result = orchestrator.execute_chain(chain_type, loan_application)
    results[chain_type.value] = result
    print(f" ✓ ({result['execution_time']:.2f}s, {result.get('total_tokens', 0)} tokens)")

### Chain Comparison

In [ ]:
# Display summary
print("\nCHAIN COMPARISON:")
print("=" * 80)

comparison = pd.DataFrame([
    {
        'Chain': ct.value,
        'Time (s)': f"{results[ct.value]['execution_time']:.2f}",
        'Tokens': results[ct.value].get('total_tokens', 0),
        'Steps': results[ct.value].get('steps_executed', results[ct.value].get('iterations', 'N/A'))
    }
    for ct in ChainType
])

print(comparison.to_string(index=False))

### Orchestrator Statistics

In [ ]:
# Orchestrator stats
print("\nORCHESTRATOR STATISTICS:")
print("=" * 80)
stats = orchestrator.get_stats()
for key, value in stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")

print("=" * 80)

## LAB SUMMARY

### Chain Patterns Mastered

| Pattern | Best For | Complexity | Token Efficiency |
|---------|----------|------------|------------------|
| **Sequential** | Linear workflows | ⭐⭐⭐☆☆ | High |
| **Parallel** | Independent tasks | ⭐⭐⭐⭐☆ | High |
| **Conditional** | Decision trees | ⭐⭐⭐⭐☆ | Highest |
| **Refinement** | Quality improvement | ⭐⭐⭐⭐☆ | Medium |

### When to Use Each Pattern

```
Use Sequential when:
- Steps have clear dependencies
- Output of step N is input to step N+1
- Example: Extract → Calculate → Assess → Recommend

Use Parallel when:
- Tasks are independent
- Speed matters
- Example: Credit check + Financial analysis + Collateral check

Use Conditional when:
- Different inputs need different processing
- Want to optimize based on criteria
- Example: Small/Medium/Large loan routing

Use Refinement when:
- Output quality is critical
- Initial attempt may be insufficient
- Example: Report generation, recommendations
```

### Production Checklist

- [x] Identify which chain pattern fits your workflow
- [x] Design clear step boundaries
- [x] Handle errors at each step
- [x] Log execution for debugging
- [x] Monitor token usage per step
- [x] Set timeouts for each step
- [x] Implement retries for failures
- [x] Test with edge cases
- [x] Measure end-to-end performance

### Key Takeaways

✓ **Sequential chains** - Simple, linear, debuggable workflows  
✓ **Parallel processing** - 3x speed improvement for independent tasks  
✓ **Conditional branching** - Route processing based on input characteristics  
✓ **Refinement loops** - Iteratively improve output quality  
✓ **Production orchestration** - Unified system for all chain types

## NEXT STEPS

**Lab 2.6: Context-Aware Q&A System (Capstone)** - Bringing all Session 2 techniques together in a complete production system.

**Apply these techniques to:**
- Complex multi-step workflows
- Loan underwriting and approval processes
- Document analysis pipelines
- Quality assurance systems
- Any task requiring multiple LLM calls with context passing

---

**End of Lab 2.5**